<a href="https://colab.research.google.com/github/Justin21523/edge-deid-studio/blob/feature%2Fadd-ner-evaluation-notebook/notebooks/06_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 一、環境準備（Cell 1–3）

In [ ]:
!pip install -q transformers datasets seqeval matplotlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.environ['HF_HOME']          = '/content/drive/MyDrive/hf_cache'
os.environ['TRANSFORMERS_CACHE'] = '/content/drive/MyDrive/hf_cache/transformers'
os.makedirs(os.environ['TRANSFORMERS_CACHE'], exist_ok=True)

In [ ]:
import torch
print("GPU available:", torch.cuda.is_available())

> **說明**：
>
> 1. `seqeval` 用來計算 NER 的 Precision/Recall/F1；`matplotlib` 用於圖表視覺化。
> 2. Drive 掛載、快取目錄設定與前幾個 notebook 保持一致，確保讀寫相同模型與資料夾。
> 3. 確認 GPU 狀態，雖然評估多數可在 CPU 上執行，但若需要大量 batch predict，可切至 GPU。

## 二、載入測試資料與模型（Cell 4–6）

In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset("conll2003")
test_dataset = raw_datasets["test"]
label_list   = raw_datasets["train"].features["ner_tags"].feature.names
print("測試集大小：", len(test_dataset), "，標籤種類：", label_list)

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_dir = "models/ner/v1.0"   # 請改為你微調並儲存 NER 模型的路徑
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model     = AutoModelForTokenClassification.from_pretrained(model_dir)
model.eval()

In [ ]:
# Tokenize 與 label align 函式（與 02_train_ner.ipynb 相同）
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True
    )
    aligned_labels = []
    for i, labels in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(labels[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        aligned_labels.append(label_ids)
    tokenized_inputs["labels"] = aligned_labels
    return tokenized_inputs

tokenized_test = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=test_dataset.column_names
)

> **說明**：
>
> * 先下載 `conll2003` Test split，並讀出 NER 標籤清單。
> * 使用與訓練時相同的 `tokenize_and_align_labels`，確保輸入與標籤映射一致。&#x20;


## 三、定義評估流程與指標（Cell 7）

In [ ]:
from transformers import DataCollatorForTokenClassification, Trainer, TrainingArguments
from seqeval.metrics import classification_report, precision_score, recall_score, f1_score

data_collator = DataCollatorForTokenClassification(tokenizer)

# 建立 Trainer 用於 predict
training_args = TrainingArguments(
    output_dir="eval_tmp",
    per_device_eval_batch_size=8,
    do_train=False,
    do_predict=True
)
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator
)

def compute_metrics(predictions, labels):
    # predictions: np.ndarray (batch_size, seq_len, num_labels)
    # labels: np.ndarray (batch_size, seq_len)
    preds = predictions.argmax(-1)
    true_labels = [
        [label_list[l] for l in label_seq if l != -100]
        for label_seq in labels
    ]
    true_preds = [
        [label_list[p] for (p, l) in zip(pred_seq, lab_seq) if l != -100]
        for pred_seq, lab_seq in zip(preds, labels)
    ]
    return {
        "precision": precision_score(true_labels, true_preds),
        "recall":    recall_score(true_labels, true_preds),
        "f1":        f1_score(true_labels, true_preds),
        "report":    classification_report(true_labels, true_preds)
    }


> **說明**：
>
> * `Trainer.predict()` 會回傳 `(predictions, labels, metrics)`。
> * 利用 `seqeval` 計算整體與每個 entity 的 P/R/F1，並產出詳細報告。&#x20;

## 四、執行預測並計算指標（Cell 8）

In [ ]:
# 真正執行前先註解，確認語法
# predictions, labels, _ = trainer.predict(tokenized_test)

# metrics = compute_metrics(predictions, labels)
# print("Overall Precision:", metrics["precision"])
# print("Overall Recall:",    metrics["recall"])
# print("Overall F1:",        metrics["f1"])
# print(metrics["report"])

> **說明**：
>
> * 將 `trainer.predict()` 註解起來，待切到 GPU 環境再啟用。
> * `compute_metrics` 將自動過濾 `-100` 標籤，對齊子詞後只評估首個 sub-token。

## 五、視覺化比較不同模型版本（Cell 9）

In [ ]:
import matplotlib.pyplot as plt

# 假設你對比了兩個版本 (v1.0 vs v1.1) 的整體 F1
versions = ["v1.0", "v1.1"]
f1_scores = [0.89, 0.91]  # 例子，替換成真實結果

plt.figure(figsize=(6,4))
plt.bar(versions, f1_scores)
plt.xlabel("Model Version")
plt.ylabel("F1 Score")
plt.title("NER Model F1 Comparison")
# plt.show()  # 在 Colab 上會自動渲染


> **說明**：
>
> * 以長條圖呈現不同版本整體 F1，只要把 `f1_scores` 改為實際讀取的結果即可。
> * 也可以畫細分 label 的條形圖，程式碼結構相同。

## 六、小結與下一步（Cell 10）

- ✅ NER 測試集上計算出了 Precision/Recall/F1 指標  
- ✅ 可視化不同模型版本表現  
- ‼️ 等到 GPU 環境後，請解開 `trainer.predict()` 的註解，並填入真實指標到可視化程式碼中  